In [2]:
from kafka import KafkaProducer
import json
from json import loads
from csv import DictReader

bootstrap_servers = ['localhost:9092']
topicname = 'weather2000'
producer = KafkaProducer(bootstrap_servers=bootstrap_servers)
producer = KafkaProducer()

limit = 5737

with open('smallerCSV/2000.csv', 'r') as new_obj:
    csv_dict_reader = DictReader(new_obj)
    for idx, row in enumerate(csv_dict_reader, 1):
        ack = producer.send(topicname, json.dumps(row).encode('utf-8'))
        metadata = ack.get()
        print(metadata.topic, metadata.partition)
        if idx == limit:
            break

weather2000 0
weather2000 0
weather2000 0
weather2000 0
weather2000 0
weather2000 0
weather2000 0
weather2000 0
weather2000 0
weather2000 0
weather2000 0
weather2000 0
weather2000 0
weather2000 0
weather2000 0
weather2000 0
weather2000 0
weather2000 0
weather2000 0
weather2000 0
weather2000 0
weather2000 0
weather2000 0
weather2000 0
weather2000 0
weather2000 0
weather2000 0
weather2000 0
weather2000 0
weather2000 0
weather2000 0
weather2000 0
weather2000 0
weather2000 0
weather2000 0
weather2000 0
weather2000 0
weather2000 0
weather2000 0
weather2000 0
weather2000 0
weather2000 0
weather2000 0
weather2000 0
weather2000 0
weather2000 0
weather2000 0
weather2000 0
weather2000 0
weather2000 0
weather2000 0
weather2000 0
weather2000 0
weather2000 0
weather2000 0
weather2000 0
weather2000 0
weather2000 0
weather2000 0
weather2000 0
weather2000 0
weather2000 0
weather2000 0
weather2000 0
weather2000 0
weather2000 0
weather2000 0
weather2000 0
weather2000 0
weather2000 0
weather2000 0
weathe

In [1]:
from kafka import KafkaConsumer, KafkaProducer
import json
from json import loads
from csv import DictReader
import sys

bootstrap_servers = ['localhost:9092']

topicname = 'weatherdump'
consumer = KafkaConsumer(topicname, bootstrap_servers=bootstrap_servers, auto_offset_reset='earliest', value_deserializer =lambda x:loads(x.decode('utf-8')))

try:
    for message in consumer:
        print(message.value)
except KeyboardInterrupt:  
    sys.exit() 


{'date': '2000-05-07', 'hour': '00:00', 'prcp': '-9999.0', 'stp': '-9999.0', 'smax': '-9999.0', 'smin': '-9999.0', 'gbrd': '-9999', 'temp': '-9999.0', 'dewp': '-9999.0', 'tmax': '-9999.0', 'tmin': '-9999.0', 'dmax': '-9999.0', 'dmin': '-9999', 'hmax': '-9999', 'hmin': '-9999', 'hmdy': '-9999', 'wdct': '-9999.0', 'gust': '-9999.0', 'wdsp': 'SE', 'prov': 'RJ', 'wsnm': 'ECOLOGIA AGRICOLA', 'inmen': 'A601', 'lat': '-22.8', 'lon': '-43.68333333', 'elvt': '33.0', 'year': '2000'}
{'date': '2000-05-07', 'hour': '01:00', 'prcp': '-9999.0', 'stp': '-9999.0', 'smax': '-9999.0', 'smin': '-9999.0', 'gbrd': '-9999', 'temp': '-9999.0', 'dewp': '-9999.0', 'tmax': '-9999.0', 'tmin': '-9999.0', 'dmax': '-9999.0', 'dmin': '-9999', 'hmax': '-9999', 'hmin': '-9999', 'hmdy': '-9999', 'wdct': '-9999.0', 'gust': '-9999.0', 'wdsp': 'SE', 'prov': 'RJ', 'wsnm': 'ECOLOGIA AGRICOLA', 'inmen': 'A601', 'lat': '-22.8', 'lon': '-43.68333333', 'elvt': '33.0', 'year': '2000'}
{'date': '2000-05-07', 'hour': '02:00', 'prc

AttributeError: 'tuple' object has no attribute 'tb_frame'

In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/opt/spark"

In [2]:
scala_version = '2.12'
spark_version = '3.4.0'
packages = [
    f'org.apache.spark:spark-sql-kafka-0-10_{scala_version}:{spark_version}',
    'org.apache.kafka:kafka-clients:3.4.0'
]

In [4]:
# Kafka consumer
from pyspark.sql import SparkSession
spark = SparkSession \
        .builder \
        .appName("test") \
        .config("spark.jars.packages", ",".join(packages))\
        .master("local[*]") \
        .getOrCreate()
kafka_df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "wiki") \
    .option("startingOffsets", "earliest") \
    .load()

print(kafka_df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)"))
# query = kafka_df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)") \
#     .writeStream \
#     .format("console") \
#     .option("checkpointLocation", "dis/checkpoint") \
#     .start()

# query.awaitTermination()

DataFrame[key: string, value: string]


In [6]:
kafka_df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [11]:
readable = kafka_df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")

query = readable.writeStream  \
    .outputMode("append")  \
    .format("console")  \
    .option("truncate", "False")  \
    .start()
    
query.awaitTermination()

23/04/25 03:48:42 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-01a6adfc-dc4f-405d-b0e0-18ebd9d5b486. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/04/25 03:48:42 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+----+---------------------------------------------------------------------------------------+
|key |value                                                                                  |
+----+---------------------------------------------------------------------------------------+
|null|{"prev": "Greek_language", "curr": "Aristotle", "type": "link", "n": "59"}             |
|null|{"prev": "Exaggeration", "curr": "Aristotle", "type": "link", "n": "11"}               |
|null|{"prev": "Euclid", "curr": "Aristotle", "type": "link", "n": "39"}                     |
|null|{"prev": "Lucid_dream", "curr": "Aristotle", "type": "link", "n": "36"}                |
|null|{"prev": "Oceanography", "curr": "Aristotle", "type": "link", "n": "19"}               |
|null|{"prev": "Thespis", "curr": "Aristotle", "type": "link", "n": "34"}                    |
|null|{"prev": "Animal_rights", "curr": "Aristot

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/home/dis/.local/lib/python3.8/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/dis/.local/lib/python3.8/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.8/socket.py", line 669, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [12]:
kafka_df.select("value")

DataFrame[value: binary]

In [19]:
from pyspark.sql.functions import col, from_json
from pyspark.sql.types import StructType, StringType, StructField, IntegerType
import pandas as pd 

schema = StructType([
    StructField("prev", StringType(), True),
    StructField("curr", StringType(), True),
    StructField("link", StringType(), True),
    StructField("n", IntegerType(), True)
])

df = pd.DataFrame()
df = readable.select(col("key").cast("string"),
  from_json(col("value").cast("string"), schema))

df


DataFrame[key: string, from_json(CAST(value AS STRING)): struct<prev:string,curr:string,link:string,n:int>]

In [23]:
query = df.writeStream  \
    .outputMode("append")  \
    .format("console")  \
    .option("truncate", "False")  \
    .start() 

query.awaitTermimation()

23/04/25 04:24:32 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-b7186966-899c-49d3-ba20-ea982cc5764f. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/04/25 04:24:32 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


AttributeError: 'StreamingQuery' object has no attribute 'awaitTermimation'

-------------------------------------------
Batch: 0
-------------------------------------------
+----+----------------------------------------------------+
|key |from_json(CAST(value AS STRING))                    |
+----+----------------------------------------------------+
|null|{Greek_language, Aristotle, null, null}             |
|null|{Exaggeration, Aristotle, null, null}               |
|null|{Euclid, Aristotle, null, null}                     |
|null|{Lucid_dream, Aristotle, null, null}                |
|null|{Oceanography, Aristotle, null, null}               |
|null|{Thespis, Aristotle, null, null}                    |
|null|{Animal_rights, Aristotle, null, null}              |
|null|{History_of_economic_thought, Aristotle, null, null}|
|null|{Greek_words_for_love, Aristotle, null, null}       |
|null|{Praxis_(process), Aristotle, null, null}           |
+----+----------------------------------------------------+

